In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.prebuilt import create_react_agent
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.graph import END
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from typing import Dict
from config import Settings
import os
import uuid

from pydantic import BaseModel, Field
import operator
from typing import Annotated, List, Tuple, Union
from typing_extensions import TypedDict

### Define OpenAI Models

In [2]:
api_key = Settings.api_key
VECTOR_DIR = 'vectorize'
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=api_key)

In [3]:
llm = ChatOpenAI(model="gpt-4o", api_key=api_key)

SYSTEM_TEMPLATE = """
    Answer the user's questions based on the below context. 
    If the context doesn't contain any relevant information to the questions, don't make someting up
    and just reply information cannot be fount:
    <context>
    {context}
    </context>
    """

### Define Tools

We first define the vectorstore where we will be storing our memories. Memories will be stored as embeddings and later looked up based on the converstation context. We are using an in-memory vectorstore. 

In [ ]:
recall_vector_store = InMemoryVectorStore(embeddings)

def get_user_id(config: RunnableConfig) -> str:

    user_id = config["configurable"].get("user_id")
    if user_id in None:
        raise ValueError("User ID needs to be provided to save memory")
    return user_id

@tool
def save_recall_memory(memory: str, config: RunnableConfig) -> str:

    "Save memory to vectorstore for later semantic search retrieval."
    user_id = get_user_id(config)
    document = Document(page_content=memory, id=str(uuid.uuid4()), metadata={"user_id": user_id})
    recall_vector_store.add_document([document])

@tool
def search_recall_memories(query: str, config: RunnableConfig) -> List[str]:
    """Search for relevant memories."""

    user_id = get_user_id(config)

    def _filter_function(doc: Document) -> bool:
        return doc.metadata.get("user_id") == user_id

    documents = recall_vector_store.similarity_search(query, k=3, filter=_filter_function)
    return [document.page_content for document in documents]


In [4]:
def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


@tool
def contextualQA(earnings_question: str, company_name: str) -> str:

    """
    Identifies the relevant context in the earning call transcripts to the user question

    This tool searches in the earnings calls transcript documents and extract financial information
    such as net income, REvenue, EBITDA and etc.

    Parameters:
    - earnings_question: The questions asked by the user
    - the company for which we need to answer the question. Company names should always be lowercase

    Returns:
    - A string with the context that contain the answer to the earnings question.
    """
    vector_store = FAISS.load_local(VECTOR_DIR + "/" + company_name, 
                                    embeddings, 
                                    allow_dangerous_deserialization=True)
    retriever = vector_store.as_retriever()
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", SYSTEM_TEMPLATE),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    qa_chain = create_stuff_documents_chain(llm, qa_prompt)
    retrieval_chain = RunnablePassthrough.assign(
    context = parse_retriever_input | retriever).assign(answer=qa_chain)
    response = retrieval_chain.invoke(
        {
            "messages": [HumanMessage(content=earnings_question)]
        }
    )
    return response["answer"]


In [ ]:
tools = [save_recall_memory, search_recall_memories, contextualQA]



In [ ]:
# Define VectorStore for memories